# Trade Data Analysis

### Section 1: Import Libraries and Load Data

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import json
import ast

# load the dataset
df = pd.read_csv('TRADES_CopyTr_90D_ROI.csv', on_bad_lines='skip')

# display the first few rows of the dataset
df.head()

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


### Section 2: Data Preprocessing

In [ ]:
# Remove rows with missing Trade_History
df_clean = df.dropna(subset=['Trade_History'])

# parse Trade_History into JSON format
df_clean['Trade_History'] = df_clean['Trade_History'].apply(ast.literal_eval)

# display the first few parsed entries
print(df_clean['Trade_History'].head())

0    [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1    [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2    [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3    [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4    [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...
Name: Trade_History, dtype: object


C:\Users\saksh\AppData\Local\Temp\ipykernel_8400\3883334104.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Trade_History'] = df_clean['Trade_History'].apply(ast.literal_eval)


### Section 3: Extracting Trade Details

In [ ]:
# Create an empty list to store trade data
trade_data_list = []

# loop through each row and extract trade details
for idx, row in df_clean.iterrows():
    port_id = row['Port_IDs']
    trade_history = row['Trade_History']

    for trade in trade_history:
        trade_data_list.append({
            'Port_ID': port_id,
            'time': trade.get('time'),
            'symbol': trade.get('symbol'),
            'side': trade.get('side'),
            'price': trade.get('price'),
            'quantity': trade.get('quantity'),
            'realizedProfit': trade.get('realizedProfit', 0),
            'positionSide': trade.get('positionSide')
        })

# convert the list to a DataFrame
trade_data_df = pd.DataFrame(trade_data_list)

# convert 'time' to datetime format
trade_data_df['time'] = pd.to_datetime(trade_data_df['time'], unit='ms')

# display the extracted data
print(trade_data_df.head())

               Port_ID                time    symbol side      price  \
0  3925368433214965504 2024-06-20 16:07:36   SOLUSDT  BUY  132.53700   
1  3925368433214965504 2024-06-20 16:06:58  DOGEUSDT  BUY    0.12182   
2  3925368433214965504 2024-06-20 16:06:58  DOGEUSDT  BUY    0.12182   
3  3925368433214965504 2024-06-20 16:06:56  DOGEUSDT  BUY    0.12182   
4  3925368433214965504 2024-06-20 16:06:56  DOGEUSDT  BUY    0.12182   

     quantity  realizedProfit positionSide  
0  1988.05500             0.0         LONG  
1  1398.98088             0.0         LONG  
2   197.47022             0.0         LONG  
3    16.56752             0.0         LONG  
4    92.21774             0.0         LONG  


### Section 4: Calculation of Metrics

In [14]:
# Group data by Port_ID
grouped = trade_data_df.groupby('Port_ID')

# Define a function to calculate key metrics
def calculate_metrics(group):
    roi = (group['realizedProfit'].sum() / (group['price'] * group['quantity']).sum()) * 100 if (group['price'] * group['quantity']).sum() > 0 else 0
    pnl = group['realizedProfit'].sum()
    win_rate = (group['realizedProfit'] > 0).mean() * 100
    total_trades = len(group)
    sharpe = (group['realizedProfit'].mean() - 0.01) / group['realizedProfit'].std() if group['realizedProfit'].std() != 0 else 0
    
    return pd.Series({
        'ROI': roi,
        'PnL': pnl,
        'Win_Rate': win_rate,
        'Total_Trades': total_trades,
        'Sharpe_Ratio': sharpe
    })

# Apply the function to calculate metrics for each account
metrics_df = grouped.apply(calculate_metrics)

# Normalize metrics (optional)
cols_to_normalize = ['ROI', 'PnL', 'Win_Rate', 'Sharpe_Ratio']
for col in cols_to_normalize:
    min_val = metrics_df[col].min()
    max_val = metrics_df[col].max()
    if max_val != min_val:
        metrics_df[f'{col}_norm'] = (metrics_df[col] - min_val) / (max_val - min_val)
    else:
        metrics_df[f'{col}_norm'] = metrics_df[col] / max_val if max_val != 0 else 0

# Calculate composite score (with equal weights)
metrics_df['Score'] = (
    metrics_df['ROI_norm'] * 0.25 +
    metrics_df['PnL_norm'] * 0.25 +
    metrics_df['Win_Rate_norm'] * 0.25 +
    metrics_df['Sharpe_Ratio_norm'] * 0.25
)

# Calculate rank (1 is best)
metrics_df['Rank'] = metrics_df['Score'].rank(ascending=False, method='min')

# Extract top 20 accounts
top_20_accounts = metrics_df.sort_values('Rank').head(20)

# Display results
print(top_20_accounts[['ROI', 'PnL', 'Win_Rate', 'Total_Trades', 'Score', 'Rank']].round(2))

                       ROI       PnL  Win_Rate  Total_Trades  Score  Rank
Port_ID                                                                  
3977234346014419201  36.53   2427.29     39.76          83.0   0.54   1.0
3826087012661391104   0.00    532.66     58.33         108.0   0.47   2.0
3986814617275053313   0.09  16337.46     78.22        3554.0   0.46   3.0
3956048468100538880   0.00   1373.56     71.43          28.0   0.46   4.0
3891020560590657281   6.80   2856.30     64.76         437.0   0.46   5.0
3999240873283311617   0.10  42574.47     52.32        4522.0   0.43   6.0
3983074113875692800  18.79   2424.83     32.56          43.0   0.42   7.0
4039279455324236544   1.84   1038.81     55.35         327.0   0.41   8.0
4020204877254599680   0.00  71998.86     32.94        6050.0   0.40   9.0
3886752488982104320   0.00   7195.18     74.54        1249.0   0.39  10.0
4022641794255717633   0.35   2176.83     43.52        6052.0   0.39  11.0
3943533600390906881   0.00   2361.42  

C:\Users\saksh\AppData\Local\Temp\ipykernel_8400\2914380992.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics_df = grouped.apply(calculate_metrics)


### Section 5: Saving the Results

In [15]:
# Save full metrics to CSV
metrics_df.to_csv(r'metrics.csv', index=False)

# Save top 20 accounts to CSV
top_20_accounts.to_csv(r'top_20_accounts.csv', index=False)